## dataClay connection

In [1]:
from dataclay import Client
from dataclay.config import session_var
client = Client(proxy_host="127.0.0.1", dataset="admin")
client.start()

/home/alex/BSC/otlp-dataclay-bridge/venv/lib/python3.11/site-packages/pydantic_settings/sources.py:376: UserWarning: directory "/run/secrets" does not exist
  warnings.warn(f'directory "{self.secrets_path}" does not exist')
INFO:dataclay.client.api:Starting client runtime


Starting event loop in new thread


INFO:dataclay.backend.client:SSL not configured


## Prepare bridge configuration

In [2]:
import operator
from dataclay.exceptions import DataClayException
from model.config import BridgeConfiguration, ResourceConfiguration, MatchRule

rules: list[MatchRule] = [
    ("service.name", operator.eq, "otelcol"),
    ("service.instance.id", operator.eq, "127.0.0.1:8888")
]

try:
    bc = await BridgeConfiguration.a_get_by_alias("bridge_config")
    print("BridgeConfiguration is already persistent in dataClay.")
except DataClayException:
    print("Creating new BridgeConfiguration")
    
    bc = BridgeConfiguration()
    rc = ResourceConfiguration("otel-telemetry-demo", rules)
    rc.add_metric("otelcol_process_cpu_seconds")
    rc.add_metric("otelcol_rpc_client_duration")
    rc.add_metric("otelcol_rpc_client_requests_per_rpc")
    bc.set_res_config(rc)
    await bc.a_make_persistent(alias="bridge_config")


Creating new BridgeConfiguration


In [3]:
bc.resource_configurations

{'otel-telemetry-demo': <model.config.ResourceConfiguration instance with ObjectID=c733dea7-637c-46dd-9240-ff82012737bb>}